In [101]:
# !pip install keras
# pip install keras
# pip install tensorflow
# !pip3 install opencv-python
# !pip3 install opencv-python
# pip install opencv-contrib-python

In [239]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import os
import random
os.environ['KMP_DUPLICATE_LIB_OK']='True' #This prevents kernel shut down due to xgboost conflict

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
from sklearn.dummy import DummyClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D, Activation
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.wrappers import scikit_learn
from tensorflow.keras.callbacks import EarlyStopping

import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

import tensorflow as tf
import cv2

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data

In [242]:
X_in = open('Data/pickles/X.pickle', 'rb')
X = pickle.load(X_in)
y_in = open('Data/pickles/y.pickle', 'rb')
y = pickle.load(y_in)

Xtest_in = open('Data/pickles/X_test.pickle', 'rb')
X_test = pickle.load(Xtest_in)
ytest_in = open('Data/pickles/y_test.pickle', 'rb')
y_test = pickle.load(ytest_in)

Xvalid_in = open('Data/pickles/X_valid.pickle', 'rb')
X_valid = pickle.load(Xvalid_in)
yvalid_in = open('Data/pickles/y_valid.pickle', 'rb')
y_valid = pickle.load(yvalid_in)

## Baseline Model

In [243]:
dummy = DummyClassifier(strategy='most_frequent')
dummy.fit(X, y)
baseline_acc = round(dummy.score(X, y), 2) * 100
print('Baseline Accuracy: {0}%'.format(baseline_acc))

Baseline Accuracy: 74.0%


## Model 1

In [222]:
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape = X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32)

163/163 [==============================] - 62s 372ms/step - loss: 0.3088 - accuracy: 0.8744


In [223]:
model = Sequential()

model.add(    Conv2D(64, (3,3), input_shape = X.shape[1:])    )
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X, y, batch_size=32, validation_split=0.1, epochs=3)

Epoch 1/3
147/147 [==============================] - 53s 352ms/step - loss: 0.2487 - accuracy: 0.9018 - val_loss: 0.0989 - val_accuracy: 0.9464
Epoch 2/3
147/147 [==============================] - 51s 348ms/step - loss: 0.1159 - accuracy: 0.9561 - val_loss: 0.0995 - val_accuracy: 0.9598
Epoch 3/3
147/147 [==============================] - 57s 389ms/step - loss: 0.0812 - accuracy: 0.9715 - val_loss: 0.1329 - val_accuracy: 0.9483


In [224]:
model.evaluate(X_test, y_test)

20/20 [==============================] - 2s 72ms/step - loss: 0.9154 - accuracy: 0.7750


[0.9153845906257629, 0.7749999761581421]

## Model 2

In [186]:
model2 = Sequential()
model2.add(Flatten())
model2.add(Dense(20, activation='relu', input_shape=X.shape[1:])) # 2 hidden layers
model2.add(Dense(7, activation='relu'))
model2.add(Dense(5, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))

model2.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model2.fit(X, y, epochs=3, batch_size=32)

Epoch 1/3
163/163 [==============================] - 1s 4ms/step - loss: 0.3056 - accuracy: 0.8767
Epoch 2/3
163/163 [==============================] - 0s 3ms/step - loss: 0.1839 - accuracy: 0.9354
Epoch 3/3
163/163 [==============================] - 1s 4ms/step - loss: 0.1674 - accuracy: 0.9298


In [187]:
model.evaluate(X_test, y_test)

20/20 [==============================] - 2s 88ms/step - loss: 0.3123 - accuracy: 0.8797


[0.3122536540031433, 0.879687488079071]

## Model 3

In [197]:
model3 = Sequential()
model3.add(Flatten())
model3.add(Dense(20, activation='relu', input_shape=X.shape[1:])) # 2 hidden layers
model3.add(Dense(7, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(6, activation='relu'))
model3.add(Dense(5, activation='relu'))
model3.add(Dense(1, activation='sigmoid'))

model3.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model3.fit(X, y, epochs=3, batch_size=32)

Epoch 1/3
163/163 [==============================] - 1s 3ms/step - loss: 0.4947 - accuracy: 0.7418
Epoch 2/3
163/163 [==============================] - 0s 3ms/step - loss: 0.3708 - accuracy: 0.8533
Epoch 3/3
163/163 [==============================] - 0s 3ms/step - loss: 0.3250 - accuracy: 0.8967


In [198]:
model3.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.4082 - accuracy: 0.8844


[0.40822935104370117, 0.8843749761581421]

## Model 4

In [200]:
model4 = Sequential()
model4.add(Flatten())
model4.add(Dense(20, activation='relu', input_shape=X.shape[1:]))
model4.add(Dropout(0.2))
model4.add(Dense(7, activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(6, activation='relu'))
model4.add(Dense(5, activation='relu'))
model4.add(Dense(1, activation='sigmoid'))

model4.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model4.fit(X, y, epochs=7, batch_size=32)

Epoch 1/7
163/163 [==============================] - 2s 3ms/step - loss: 0.5059 - accuracy: 0.7103
Epoch 2/7
163/163 [==============================] - 1s 4ms/step - loss: 0.3431 - accuracy: 0.8209
Epoch 3/7
163/163 [==============================] - 1s 3ms/step - loss: 0.3207 - accuracy: 0.8855
Epoch 4/7
163/163 [==============================] - 1s 4ms/step - loss: 0.2948 - accuracy: 0.9015
Epoch 5/7
163/163 [==============================] - 1s 4ms/step - loss: 0.2675 - accuracy: 0.9074
Epoch 6/7
163/163 [==============================] - 1s 3ms/step - loss: 0.2465 - accuracy: 0.9210
Epoch 7/7
163/163 [==============================] - 1s 5ms/step - loss: 0.2447 - accuracy: 0.9072


In [201]:
model4.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.4944 - accuracy: 0.8500


[0.49444133043289185, 0.8500000238418579]

## Model 5

In [204]:
model5 = Sequential()
model5.add(Flatten())
model5.add(Dense(20, activation='relu', input_shape=X.shape[1:]))
model5.add(Dense(7, activation='relu'))
model5.add(Dropout(0.2))
model5.add(Dense(7, activation='relu'))
model5.add(Dropout(0.2))
model5.add(Dense(6, activation='relu'))
model5.add(Dense(5, activation='relu'))
model5.add(Dense(1, activation='sigmoid'))

model5.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model5.fit(X, y, epochs=6, batch_size=32)

Epoch 1/6
163/163 [==============================] - 1s 3ms/step - loss: 0.5178 - accuracy: 0.7312
Epoch 2/6
163/163 [==============================] - 1s 4ms/step - loss: 0.3723 - accuracy: 0.7912
Epoch 3/6
163/163 [==============================] - 1s 4ms/step - loss: 0.2884 - accuracy: 0.9279
Epoch 4/6
163/163 [==============================] - 1s 3ms/step - loss: 0.2568 - accuracy: 0.9317
Epoch 5/6
163/163 [==============================] - 0s 3ms/step - loss: 0.2357 - accuracy: 0.9358
Epoch 6/6
163/163 [==============================] - 0s 3ms/step - loss: 0.2141 - accuracy: 0.9413


In [205]:
model5.evaluate(X_test, y_test)

20/20 [==============================] - 0s 2ms/step - loss: 0.4182 - accuracy: 0.8734


[0.41818422079086304, 0.8734375238418579]

In [207]:
import xgboost as xgb
modelxgb = xgb.XGBClassifier(gamma=0.5, 
                             learning_rate=0.01, 
                             max_depth=4, 
                             objective='multi:softmax', 
                             silent=False,
                             subsample=0.8,
                             reg_alpha=0.5
)

In [235]:
cv_results = cross_val_score(modelxgb, X, y, cv=2, scoring='accuracy')
cv_results

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.8/site-packages/xgboost/core.py", line 436, in inner_f
    return f(**kwargs)
  File "/op

array([nan, nan])

In [237]:
modelxgb.fit(X,y)

ValueError: Please reshape the input data X into 2-dimensional matrix.

In [215]:
X.shape

(5216, 80, 80, 1)

In [216]:
X_train = X.reshape(5216, 6400)

In [221]:
# transform the dataset
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
X_smote, y_smote = oversample.fit_resample(X, y)

ValueError: Found array with dim 4. Estimator expected <= 2.